# Do over

Last time, we tried to use pdfminer to mine the [Arpanet Directory](https://www.google.com/books/edition/ARPANET_Directory/AHo-AQAAIAAJ?hl=en&gbpv=1&dq=arpanet+directory&printsec=frontcover) we found on Google Books. We learned some valuable things, such as that the pdf we are working with actually contains _three_ years worth of directories, starting with 1978. But we ran into [issues](https://github.com/pdfminer/pdfminer.six/issues/656) with out of place text while using pdfminer. So instead I tried the simple expedient of `ctrl + a` and it worked like a charm.

I also decided it would be helpful to subdivde the full pdf even further, extracting out only the HOST ACRONYMS AND NETWORKING LIASONS table from 1978. The pdf for this table is found [here](/files/pdfs/arpanet-directory_host-acronyms-1978.pdf). The text version is [here](/files/text/hosts-1978.txt).

In [37]:
import re
import pandas as pd

In [95]:
phone_number_regex = '(\( [0-9]{3} \) [0-9]{3}-[0-9]{4})'
page_number_regex = r'\n[0-9]{3}\n'
page_headers = ('HOST ACRONYMS\nACRONYM ADDR . TYPE SPONSOR LIAISON and SITE ADDRESS\n( Dec )', 
                'ARPANET DIRECTORY HOST ACRONYMS\nNIC 46099 Dec. 1978\nACRONYM ADDR . TYPE SPONSOR LIAISON and SITE ADDRESS\n( Dec )\n')
column_names = ['acronym', 'host_address', 'type', 'sponsor', 'liason_name', 'liason_email', 'physical_address']

We start by reading the file into a string and processing it:

1. Remove the first lines (up to and including the column names, which we will not use), and the last line which is a page number.
2. Replace the headers of each page with empty strings. There are two different formats of page headers to deal with.
3. Replace page numbers with empty strings.
3. Split the string, using the phone numbers as a sepator, but keeping the separators. You can do this by enclosing the separator in a capture group.
4. Zip the odd and even entries in this list of strings into a list of tuples, corresponding to the entries.

And that'll do. We will leave the phone numbers as separate strings inside the tuple.

In [111]:
with open('text/hosts-1978.txt') as hosts:
    data = ''.join(hosts.readlines()[5:-1])
    for s in page_headers: data = data.replace(s, '')
    data = re.sub(page_number_regex, '', data, 0, re.DOTALL)
    print(repr(data))
#     entries = re.split(phone_number_regex, data, 0, re.DOTALL)
#     entries = list(zip(entries[::2], entries[1::2]))

"ACCAT - TIP 2/35 TIP ARPA Brennan , Jack ( STEPHENSON@ISI )\nNaval Ocean Systems Center\nCode 722\n217 Catalina Blvd. San Diego , California 92152\n( 714 ) 225-2871\nAFWL 0/48 SERVER , AFSC\nLimited\nHavens , Martin ( AFWL@I4 - TENEX )\nAir Force Weapons Laboratory\nKirtland Air Force Base Albuquerque , New Mexico 87117 ( 505 ) 264-0319\nAFWL - TIP 2/48 TIP AFSC Maull , Roy ( MAULL@BBN - TENEX )\nAir Force Weapons Lab./ADPT Kirtland Air Force Base ,\nNew Mexico 87117\n( 505 ) 264-2581\n[ ALMSA - TIP] 2/61 TIP ARMY Nelson , Steve ( JAMES @ BBNB )\nCommander Army Communications Command\nAttn : Steve Nelson , CCNC - STL - S\nSt. Louis , Missouri 63188\nAMES - 11 3/16 USER ARPA Hart , James P. ( HART@AMES - 67 )\nNASA Ames Research Center\nNetwork Graphics Group Mail Stop 233-9 Moffett Field , California 94035\n( 415 ) 965-6629\nAMES - 67 0/16 SERVER ARPA Hathaway , Wayne ( HATHAWAY@AMES - 67 )\nNASA Ames Research Center\nComputation Division\nMail Stop 233-9 Moffett Field , California 94

In [110]:
for i in range(10):
    print(entries[i][0])

ACCAT - TIP 2/35 TIP ARPA Brennan , Jack ( STEPHENSON@ISI )
Naval Ocean Systems Center
Code 722
217 Catalina Blvd. San Diego , California 92152


AFWL 0/48 SERVER , AFSC
Limited
Havens , Martin ( AFWL@I4 - TENEX )
Air Force Weapons Laboratory
Kirtland Air Force Base Albuquerque , New Mexico 87117 

AFWL - TIP 2/48 TIP AFSC Maull , Roy ( MAULL@BBN - TENEX )
Air Force Weapons Lab./ADPT Kirtland Air Force Base ,
New Mexico 87117


[ ALMSA - TIP] 2/61 TIP ARMY Nelson , Steve ( JAMES @ BBNB )
Commander Army Communications Command
Attn : Steve Nelson , CCNC - STL - S
St. Louis , Missouri 63188
AMES - 11 3/16 USER ARPA Hart , James P. ( HART@AMES - 67 )
NASA Ames Research Center
Network Graphics Group Mail Stop 233-9 Moffett Field , California 94035


AMES - 67 0/16 SERVER ARPA Hathaway , Wayne ( HATHAWAY@AMES - 67 )
NASA Ames Research Center
Computation Division
Mail Stop 233-9 Moffett Field , California 94035 

AMES - TIP 2/16 TIP ARPA Jones , Bill ( JONES@I4 - TENEX )
NASA Ames Research Ce